In [1]:
import pandas as pd
import numpy as np
import random
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import scipy.optimize
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from math import sqrt

In [2]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols)
n_users = users.shape[0]
print('Number of users:', n_users)
#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('ua.test', sep='\t', names=r_cols)
rate_train = ratings_base.values

Number of users: 943


In [3]:
def Myrecommend():
    def normalizeRatings(myY, myR):
        # Tính trung bình các đánh giá
        Ymean = np.sum(myY, axis=1) / np.sum(myR, axis=1)
        Ymean = Ymean.reshape((Ymean.shape[0], -1))
        return myY - Ymean, Ymean

    def flattenParams(myX, myTheta):
        return np.concatenate((myX.flatten(), myTheta.flatten()))

    def reshapeParams(flattened_XandTheta, mynm, mynu, mynf):
        assert flattened_XandTheta.shape[0] == int(mynm * mynf + mynu * mynf)
        reX = flattened_XandTheta[: int(mynm * mynf)].reshape((mynm, mynf))
        reTheta = flattened_XandTheta[int(mynm * mynf) :].reshape((mynu, mynf))
        return reX, reTheta

    def cofiCostFunc(myparams, myY, myR, mynu, mynm, mynf, mylambda=0.0):
        myX, myTheta = reshapeParams(myparams, mynm, mynu, mynf)
        term1 = myX.dot(myTheta.T)
        term1 = np.multiply(term1, myR)
        cost = 0.5 * np.sum(np.square(term1 - myY))
        # Thêm phần regularization
        cost += (mylambda / 2.0) * np.sum(np.square(myTheta))
        cost += (mylambda / 2.0) * np.sum(np.square(myX))
        return cost

    def cofiGrad(myparams, myY, myR, mynu, mynm, mynf, mylambda=0.0):
        myX, myTheta = reshapeParams(myparams, mynm, mynu, mynf)
        term1 = myX.dot(myTheta.T)
        term1 = np.multiply(term1, myR)
        term1 -= myY
        Xgrad = term1.dot(myTheta)
        Thetagrad = term1.T.dot(myX)
        # Thêm phần regularization
        Xgrad += mylambda * myX
        Thetagrad += mylambda * myTheta
        return flattenParams(Xgrad, Thetagrad)

    
    mynu = df_train.user_name.nunique()
    mynm = df_train.product_id.nunique()
    mynf = 10
    Y = np.zeros((mynm, mynu))

    user_to_column = {user_name: idx for idx, user_name in enumerate(df['user_name'].unique())}
    product_to_row = {product: idx for idx, product in enumerate(df['product_id'].unique())}

    for row in df.itertuples():
        Y[product_to_row[row.product_id], user_to_column[row.user_name]] = row.rating

    R = (Y != 0).astype(int)

    Ynorm, Ymean = normalizeRatings(Y, R)
    X = np.random.rand(mynm, mynf)
    Theta = np.random.rand(mynu, mynf)
    myflat = flattenParams(X, Theta)

    result = scipy.optimize.minimize(
        fun=cofiCostFunc,
        x0=myflat,
        args=(Ynorm, R, mynu, mynm, mynf, 3),
        method="TNC",
        jac=cofiGrad,
        options={"maxiter": 300},
    )

    resX, resTheta = reshapeParams(result.x, mynm, mynu, mynf)
    prediction_matrix = resX.dot(resTheta.T)
    return prediction_matrix, Ymean, product_to_row, user_to_column

In [4]:
class uuCF(object):
    def __init__(self, Y_data, k, sim_func=cosine_similarity):
        self.Y_data = np.array(Y_data)
        self.k = k
        self.sim_func = sim_func
        self.Ybar = None
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1

    def fit(self):
        users = self.Y_data[:, 0]
        self.Ybar = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in range(self.n_users):
            ids = np.where(users == n)[0].astype(np.int32)
            ratings = self.Y_data[ids, 2]
            self.mu[n] = np.mean(ratings) if ids.size > 0 else 0
            self.Ybar[ids, 2] = ratings - self.mu[n]
        self.Ybar = sparse.coo_matrix(
            (self.Ybar[:, 2], (self.Ybar[:, 1], self.Ybar[:, 0])), (self.n_items, self.n_users)
        ).tocsr()
        self.S = self.sim_func(self.Ybar.T, self.Ybar.T)

    def pred(self, u, i):
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
        sim = self.S[u, users_rated_i]
        nns = np.argsort(sim)[-self.k :]
        nearest_s = sim[nns]
        r = self.Ybar[i, users_rated_i[nns]]
        eps = 1e-8
        return (r * nearest_s).sum() / (np.abs(nearest_s).sum() + eps) + self.mu[u]

In [5]:
# # Tạo data frame với thông tin người dùng, sản phẩm và đánh giá
# Y_data = df[['user_name', 'product_id', 'rating']].values

# # Chia dữ liệu thành tập huấn luyện và tập kiểm tra
# train_data, test_data = train_test_split(Y_data, test_size=0.2, random_state=42)

# # Tạo DataFrame cho tập huấn luyện
# df_train = pd.DataFrame(train_data, columns=['user_name', 'product_id', 'rating'])

# # Khởi tạo và chạy thử nghiệm cho Myrecommend với tham số tối ưu
# prediction_matrix, Ymean, product_to_row, user_to_column = Myrecommend()

# Tạo Y_pred cho Myrecommend trên tập kiểm tra
# Y_pred_test = np.zeros_like(test_data[:, 2])
# for idx, (user, product_id, rating) in enumerate(test_data):
#     if product_id in product_to_row and user in user_to_column:
#         Y_pred_test[idx] = prediction_matrix[product_to_row[product_id], user_to_column[user]] + Ymean[product_to_row[product_id]]
#     else:
#         Y_pred_test[idx] = np.mean(df_train['rating'])  # Giá trị trung bình nếu sản phẩm hoặc người dùng mới

# # Khởi tạo uuCF và chạy thử nghiệm với k tối ưu
# model = uuCF(train_data, k=10)
# model.fit()

# # Tạo Y_pred cho uuCF trên tập kiểm tra
# Y_pred_uuCF_test = np.zeros_like(test_data[:, 2])
# for idx, (user, product_id, rating) in enumerate(test_data):
#     Y_pred_uuCF_test[idx] = model.pred(user, product_id)

# # Tính sai số cho cả hai phương pháp trên tập kiểm tra
# mae_myrecommend_test = mean_absolute_error(test_data[:, 2], Y_pred_test)
# rmse_myrecommend_test = mean_squared_error(test_data[:, 2], Y_pred_test, squared=False)

# mae_uucf_test = mean_absolute_error(test_data[:, 2], Y_pred_uuCF_test)
# rmse_uucf_test = mean_squared_error(test_data[:, 2], Y_pred_uuCF_test, squared=False)

# print(f"MAE của Myrecommend trên tập kiểm tra: {mae_myrecommend_test}")
# print(f"MSE của Myrecommend trên tập kiểm tra: {rmse_myrecommend_test}")

# print(f"MAE của uuCF trên tập kiểm tra: {mae_uucf_test}")
# print(f"MSE của uuCF trên tập kiểm tra: {rmse_uucf_test}")

In [6]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols)
n_users = users.shape[0]
print('Number of users:', n_users)
#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('ua.test', sep='\t', names=r_cols)
rate_train = ratings_base.values
rate_test = ratings_test.values

Number of users: 943


In [7]:
rate_train[:, :2] -= 1 # since indices start from 0
rate_test[:, :2] -= 1
rs = uuCF(rate_train, k = 40)
rs.fit()
SE = 0 # squared error
# rate_train = rate_train[:, [1, 0, 2]]
# rate_test = rate_test[:, [1, 0, 2]]
rs = uuCF(rate_train, k = 40)
rs.fit()
n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in range(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2
testRMSECF = sqrt(SE/n_tests)
SE=0
for n in range(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += abs(pred - rate_test[n, 2])
testRMAECF = SE/n_tests
SE=0
n_trains=rate_train.shape[0]
for n in range(n_trains):
    pred = rs.pred(rate_train[n, 0], rate_train[n, 1])
    SE += abs(pred - rate_train[n, 2])
trainRMAECF = SE/n_trains
SE=0
for n in range(n_trains):
    pred = rs.pred(rate_train[n, 0], rate_train[n, 1])
    SE += (pred - rate_train[n, 2])**2
trainRMSECF = sqrt(SE/n_trains)


In [8]:
class MF(object):
    def __init__(self, Y, K, lam = 0.1, Xinit = None, Winit = None,learning_rate = 0.5, max_iter = 1000, print_every = 100):
        self.Y = Y # represents the utility matrix
        self.K = K
        self.lam = lam # regularization parameter
        self.learning_rate = learning_rate # for gradient descent
        self.max_iter = max_iter # maximum number of iterations
        self.print_every = print_every # print loss after each a few iters
        self.n_users = int(np.max(Y[:, 0])) + 1
        self.n_items = int(np.max(Y[:, 1])) + 1
        self.n_ratings = Y.shape[0] # number of known ratings
        self.X = np.random.randn(self.n_items, K) if Xinit is None else Xinit
        self.W = np.random.randn(K, self.n_users) if Winit is None else Winit
        self.b = np.random.randn(self.n_items) # item biases
        self.d = np.random.randn(self.n_users) # user biases]

    def loss(self):
        L = 0
        for i in range(self.n_ratings):
        # user_id, item_id, rating
            n, m, rating = int(self.Y[i,0]), int(self.Y[i,1]), self.Y[i,2]
            L += 0.5*(self.X[m].dot(self.W[:, n])\
            + self.b[m] + self.d[n] - rating)**2
        L /= self.n_ratings
        # regularization, don’t ever forget this
        return L + 0.5*self.lam*(np.sum(self.X**2) + np.sum(self.W**2))
    def updateXb(self):
        for m in range(self.n_items):
            # get all users who rated item m and corresponding ratings
            ids = np.where(self.Y[:, 1] == m)[0] # row indices of items m
            user_ids, ratings=self.Y[ids, 0].astype(np.int32),self.Y[ids, 2]
            Wm, dm = self.W[:, user_ids], self.d[user_ids]
            for i in range(30): # 30 iteration for each sub problem
                xm = self.X[m]
                error = xm.dot(Wm) + self.b[m] + dm - ratings
                grad_xm = error.dot(Wm.T)/self.n_ratings + self.lam*xm
                grad_bm = np.sum(error)/self.n_ratings
                # gradient descent
                self.X[m] -= self.learning_rate*grad_xm.reshape(-1)
                self.b[m] -= self.learning_rate*grad_bm
    def updateWd(self): # and d
        for n in range(self.n_users):
            # get all items rated by user n, and the corresponding ratings
            ids = np.where(self.Y[:,0] == n)[0] #indexes of items rated by n
            item_ids,ratings=self.Y[ids, 1].astype(np.int32), self.Y[ids, 2]
            Xn, bn = self.X[item_ids], self.b[item_ids]
            for i in range(30): # 30 iteration for each sub problem
                wn = self.W[:, n]
                error = Xn.dot(wn) + bn + self.d[n] - ratings
                grad_wn = Xn.T.dot(error)/self.n_ratings + self.lam*wn
                grad_dn = np.sum(error)/self.n_ratings
                # gradient descent
                self.W[:, n] -= self.learning_rate*grad_wn.reshape(-1)
                self.d[n] -= self.learning_rate*grad_dn    

    def fit(self):
        for it in range(self.max_iter):
            self.updateWd()
            self.updateXb()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y)
                #print('iter = %d, loss = %.4f, RMSE train = %.4f'%(it + 1,self.loss(), rmse_train))

    def pred(self, u, i):
        """
        predict the rating of user u for item i
        """
        u, i = int(u), int(i)
        pred = self.X[i, :].dot(self.W[:, u]) + self.b[i] + self.d[u]
        return max(0, min(5, pred)) # 5-scale in MoviesLen
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0] # number of test
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2
        RMSE = np.sqrt(SE/n_tests)
        return RMSE

    def evaluate_RMAE(self, rate_test):
        n_tests = rate_test.shape[0] # number of test
        AE = 0 # absolute error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            AE += abs(pred - rate_test[n, 2])
        MAE = AE / n_tests
        return MAE
    
            
            

In [9]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('ua.test', sep='\t', names=r_cols)
rate_train = ratings_base.values
rate_test = ratings_test.values
# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1
rs = MF(rate_train, K = 50, lam = .01, print_every = 5, learning_rate = 50,
max_iter = 30)
rs.fit()
# evaluate on test data
testRMSEMF = rs.evaluate_RMSE(rate_test)
testRMAEMF=rs.evaluate_RMAE(rate_test)
trainRMSEMF = rs.evaluate_RMSE(rate_train)
trainRMAEMF=rs.evaluate_RMAE(rate_train)



In [10]:
from scipy.sparse import dok_matrix
from sklearn.cluster import KMeans
from math import sqrt


In [11]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('ua.test', sep='\t', names=r_cols)
rate_train = ratings_base.values
rate_test = ratings_test.values

# Adjust user and movie IDs to start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [12]:
num_users = int(np.max(rate_train[:, 0])) + 1
num_items = int(np.max(rate_train[:, 1])) + 1

ratings_m = dok_matrix((num_users, num_items), dtype=np.float32)
for i in range(rate_train.shape[0]):
    user = rate_train[i, 0]
    item = rate_train[i, 1]
    rating = rate_train[i, 2]
    ratings_m[user, item] = rating

# Perform KMeans clustering
k = int(num_users / 10) + 2  # You can adjust this value based on your needs
kmeans = KMeans(n_clusters=k, random_state=42)
clustering = kmeans.fit(ratings_m.tocsr())

# Assign cluster labels to users
user_clusters = clustering.labels_


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [13]:
user_clusters

array([79, 23, 13, 66, 80,  6, 29, 26, 66,  6,  8, 17, 84, 19, 57, 19, 23,
       31, 66, 17,  5, 35, 19, 19, 17, 57, 66,  5, 66, 66, 66, 57, 66, 66,
       66, 66, 26, 80, 66, 13, 17, 15, 75, 80, 57, 66, 66, 17, 19, 66, 66,
       57, 66, 57, 66, 80, 57, 19, 50, 36, 66, 62, 57,  4,  8, 57, 66, 23,
       57, 80, 17, 19, 17, 13, 57, 17, 17, 66, 23, 66, 57, 19, 80, 57, 31,
       66, 35, 66, 57, 16, 19, 30, 66, 74,  4, 17, 17, 66, 32, 13, 57, 35,
       66, 57, 66, 17, 66, 23,  3, 26, 13, 13, 23, 17, 19, 10, 57,  5, 32,
       23, 17, 17, 17, 66,  8, 13, 66, 69, 66, 46, 23, 66, 66, 66,  5, 23,
       57, 17, 66, 66, 57, 66, 66, 19,  9, 13, 66, 17, 13, 23, 39,  8, 66,
       17, 66, 66, 23, 35,  1, 19, 17, 66, 66, 57, 66, 66, 66, 57, 66, 66,
       13, 66, 13, 75, 17, 57, 19, 15, 13, 17,  0, 66, 66, 14, 23, 57,  8,
       19, 67, 57, 13, 23, 57, 31, 23, 66, 26, 19, 23, 71, 41, 66, 57, 66,
       66, 13,  8, 66, 23,  8, 66, 66, 19, 19, 19, 94, 26, 17, 66, 66, 26,
       20, 57, 17, 66, 17

In [14]:
def get_cluster_recommendations(user_id, user_clusters, rate_train, top_n=8):
    user_cluster = user_clusters[user_id]
    cluster_user_ids = np.where(user_clusters == user_cluster)[0]
    
    # Aggregate ratings from the same cluster
    product_scores = {}
    for cluster_user_id in cluster_user_ids:
        user_reviews = rate_train[rate_train[:, 0] == cluster_user_id]
        for review in user_reviews:
            product_id = review[1]
            rating = review[2]
            if product_id not in product_scores:
                product_scores[product_id] = []
            product_scores[product_id].append(rating)
    
    # Compute average score for each product
    product_avg_scores = {product_id: np.mean(scores) for product_id, scores in product_scores.items()}
    recommended_products = sorted(product_avg_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    return recommended_products

def evaluate_performance(user_clusters, rate_test, rate_train):
    y_true = []
    y_pred = []
    
    for i in range(rate_test.shape[0]):
        user_id = rate_test[i, 0]
        item_id = rate_test[i, 1]
        true_rating = rate_test[i, 2]
        
        # Get cluster recommendations
        recommendations = get_cluster_recommendations(user_id, user_clusters, rate_train)
        recommended_items = [item[0] for item in recommendations]
        
        if item_id in recommended_items:
            predicted_rating = dict(recommendations).get(item_id, np.mean([r[1] for r in recommendations]))
        else:
            item_ratings = rate_train[rate_train[:, 1] == item_id][:, 2]
            predicted_rating = np.mean(item_ratings) if len(item_ratings) > 0 else np.nan
        
        y_true.append(true_rating)
        y_pred.append(predicted_rating)
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = ~np.isnan(y_pred)
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    return rmse, mae

# Evaluate performance
rmse, mae = evaluate_performance(user_clusters, rate_test, rate_train)
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")

Test RMSE: 1.0453
Test MAE: 0.8379


In [15]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('ua.test', sep='\t', names=r_cols)
rate_train = ratings_base.values
rate_test = ratings_test.values

# Adjust user and movie IDs to start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

# Filter the dataset to include only 200 users
unique_users = np.unique(rate_train[:, 0])
selected_users = np.random.choice(unique_users, 200, replace=False)

filtered_train = rate_train[np.isin(rate_train[:, 0], selected_users)]
filtered_test = rate_test[np.isin(rate_test[:, 0], selected_users)]

# Update user IDs to be continuous from 0 to 199
user_mapping = {old_id: new_id for new_id, old_id in enumerate(np.unique(filtered_train[:, 0]))}
filtered_train[:, 0] = np.vectorize(user_mapping.get)(filtered_train[:, 0])
filtered_test[:, 0] = np.vectorize(user_mapping.get)(filtered_test[:, 0])

num_users = 200
num_items = int(np.max(rate_train[:, 1])) + 1

In [16]:
# Create a sparse matrix from user reviews
ratings_m = dok_matrix((num_users, num_items), dtype=np.float32)
for i in range(filtered_train.shape[0]):
    user = filtered_train[i, 0]
    item = filtered_train[i, 1]
    rating = filtered_train[i, 2]
    ratings_m[user, item] = rating

# Perform KMeans clustering
k = int(num_users / 10) + 2  # You can adjust this value based on your needs
kmeans = KMeans(n_clusters=k, random_state=42)
clustering = kmeans.fit(ratings_m.tocsr())

# Assign cluster labels to users
user_clusters = clustering.labels_

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [17]:
def get_cluster_recommendations(user_id, user_clusters, rate_train, top_n=8):
    user_cluster = user_clusters[user_id]
    cluster_user_ids = np.where(user_clusters == user_cluster)[0]
    
    # Aggregate ratings from the same cluster
    product_scores = {}
    for cluster_user_id in cluster_user_ids:
        user_reviews = rate_train[rate_train[:, 0] == cluster_user_id]
        for review in user_reviews:
            product_id = review[1]
            rating = review[2]
            if product_id not in product_scores:
                product_scores[product_id] = []
            product_scores[product_id].append(rating)
    
    # Compute average score for each product
    product_avg_scores = {product_id: np.mean(scores) for product_id, scores in product_scores.items() if len(scores) > 0}
    recommended_products = sorted(product_avg_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    return recommended_products

def evaluate_performance(user_clusters, rate_test, rate_train):
    y_true = []
    y_pred = []
    
    for i in range(rate_test.shape[0]):
        user_id = rate_test[i, 0]
        item_id = rate_test[i, 1]
        true_rating = rate_test[i, 2]
        
        # Get cluster recommendations
        recommendations = get_cluster_recommendations(user_id, user_clusters, rate_train)
        recommended_items = [item[0] for item in recommendations]
        
        if item_id in recommended_items:
            predicted_rating = dict(recommendations).get(item_id, np.mean([r[1] for r in recommendations]))
        else:
            item_ratings = rate_train[rate_train[:, 1] == item_id][:, 2]
            predicted_rating = np.mean(item_ratings) if len(item_ratings) > 0 else np.nan
        
        y_true.append(true_rating)
        y_pred.append(predicted_rating)
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = ~np.isnan(y_pred)
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    return rmse, mae



In [18]:
train_rmse, train_mae = evaluate_performance(user_clusters, filtered_train, filtered_train)
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Train MAE: {train_mae:.4f}")


Train RMSE: 0.9744
Train MAE: 0.7682


In [19]:
# Evaluate performance
rmse, mae = evaluate_performance(user_clusters, filtered_test, filtered_train)
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")

Test RMSE: 1.1185
Test MAE: 0.8934


In [20]:
import numpy as np
import pandas as pd
from scipy.sparse import dok_matrix
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

# Load data
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv('ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('ua.test', sep='\t', names=r_cols)
rate_train = ratings_base.values
rate_test = ratings_test.values

# Adjust user and movie IDs to start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

num_users = int(np.max(rate_train[:, 0])) + 1
num_items = int(np.max(rate_train[:, 1])) + 1

# Create a sparse matrix from user reviews
ratings_m = dok_matrix((num_users, num_items), dtype=np.float32)
for i in range(rate_train.shape[0]):
    user = rate_train[i, 0]
    item = rate_train[i, 1]
    rating = rate_train[i, 2]
    ratings_m[user, item] = rating

# Apply PCA to reduce dimensionality
pca = PCA(n_components=200)  # You can adjust the number of components
ratings_m_pca = pca.fit_transform(ratings_m.tocsr().toarray())

# Perform KMeans clustering
k = int(num_users / 10) + 2  # You can adjust this value based on your needs
kmeans = KMeans(n_clusters=k, random_state=42)
clustering = kmeans.fit(ratings_m_pca)

# Assign cluster labels to users
user_clusters = clustering.labels_

def get_cluster_recommendations(user_id, user_clusters, rate_train, top_n=8):
    user_cluster = user_clusters[user_id]
    cluster_user_ids = np.where(user_clusters == user_cluster)[0]
    
    # Aggregate ratings from the same cluster
    product_scores = {}
    for cluster_user_id in cluster_user_ids:
        user_reviews = rate_train[rate_train[:, 0] == cluster_user_id]
        for review in user_reviews:
            product_id = review[1]
            rating = review[2]
            if product_id not in product_scores:
                product_scores[product_id] = []
            product_scores[product_id].append(rating)
    
    # Compute average score for each product
    product_avg_scores = {product_id: np.mean(scores) for product_id, scores in product_scores.items() if len(scores) > 0}
    recommended_products = sorted(product_avg_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    return recommended_products

def evaluate_performance(user_clusters, rate_data, rate_train):
    y_true = []
    y_pred = []
    
    for i in range(rate_data.shape[0]):
        user_id = rate_data[i, 0]
        item_id = rate_data[i, 1]
        true_rating = rate_data[i, 2]
        
        # Get cluster recommendations
        recommendations = get_cluster_recommendations(user_id, user_clusters, rate_train)
        recommended_items = [item[0] for item in recommendations]
        
        if item_id in recommended_items:
            predicted_rating = dict(recommendations).get(item_id, np.mean([r[1] for r in recommendations]))
        else:
            item_ratings = rate_train[rate_train[:, 1] == item_id][:, 2]
            predicted_rating = np.mean(item_ratings) if len(item_ratings) > 0 else np.nan
        
        y_true.append(true_rating)
        y_pred.append(predicted_rating)
    
    # Remove NaNs from y_pred and corresponding entries in y_true
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = ~np.isnan(y_pred)
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    return rmse, mae


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [21]:
# Evaluate performance on test data
test_rmse, test_mae = evaluate_performance(user_clusters, filtered_test, filtered_train)
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test MAE: {test_mae:.4f}")

# Evaluate performance on training data
train_rmse, train_mae = evaluate_performance(user_clusters, filtered_train, filtered_train)
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Train MAE: {train_mae:.4f}")

Test RMSE: 1.1166
Test MAE: 0.8912
Train RMSE: 0.9697
Train MAE: 0.7601


In [22]:
comparison_table = pd.DataFrame({
    'Model': ['MF', 'CF','Kmeans'],
    'Train RMSE': [trainRMSEMF, trainRMSECF,train_rmse],
    'Test RMSE': [testRMSEMF, testRMSECF,test_rmse],
    'Train MAE': [trainRMAEMF, trainRMAECF,train_mae],
    'Test MAE': [testRMAEMF, testRMAECF,test_mae]
})

print(comparison_table)

    Model  Train RMSE  Test RMSE  Train MAE  Test MAE
0      MF    0.911142   0.962120   0.718700  0.756727
1      CF    0.742342   0.976614   0.609017  0.769020
2  Kmeans    0.969707   1.116597   0.760080  0.891203
